In [1]:
import pandas as pd
import os
import glob

In [2]:
folder_path = '../data/raw/'

# Get a list of all the CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

## First Preprocessing

time_new and day_new for better readability
<br> no_movement column shows episodes without movement for at least 2min

In [3]:
for file in csv_files:
    df = pd.read_csv(file)
    filename = os.path.splitext(os.path.basename(file))[0]
    df.columns = df.columns.str.lower()
    df["no_movement"] =""
    day_old = 0
    week_number = 0

    df["hour"] = df["time"].astype(int)
    df["minute"] = (60 * (df["time"]-df["hour"])).astype(int)
    df["seconds"] = (60*((60 * (df["time"]-df["hour"])) - df["minute"])).astype(int)

    # Convert the columns to a common data type
    df['hour'] = df['hour'].astype('int')
    df['minute'] = df['minute'].astype('int')
    df['seconds'] = df['seconds'].astype('int')

    # Combine the columns into a single string
    df['time_new'] = df['hour'].astype('str') + ':' + df['minute'].astype('str') + ':' + df['seconds'].astype('str')

    df["day_new"] = ""

    for i in range(len(df)):
        
        df.loc[i, "day_new"] = df["day"][i]%5 
        
        #Add weeks
        day_new = df["day"][i]%5
        # i+1 < i  -> week number++
        if(day_new<day_old):
            week_number = week_number + 1
        day_old = df["day"][i]%5
        df.loc[df.index == i, 'week_num'] = week_number
        
        
        # static boolean
        if(i < len(df)-1): 
            duration = (df["time"][i+1] - df["time"][i]) * 60
            if (duration >= 2):
                df.loc[i, "no_movement"] = True
        
    df = df[["day", "time", "time_new", "day_new", "week_num", "x", "y", "z", "no_movement"]]
    df.to_csv("../data/preprocessed/" + filename + "_prep.csv", index=False)